In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import torchvision.transforms as T
from torchvision.models.segmentation import deeplabv3_resnet101

def process_image(image_path, model, device='cpu'):
    try:
        # Cargar y preparar imagen
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"No se pudo cargar la imagen: {image_path}")
        
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Detectar carretera y acera
        road_mask, sidewalk_mask = detect_road_and_sidewalk(image_rgb)
        
        # Preparar para DeepLab
        transform = T.Compose([
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        input_tensor = transform(image_rgb).unsqueeze(0)
        
        with torch.no_grad():
            output = model(input_tensor)['out']
            probs = torch.softmax(output, dim=1)
        
        semantic_mask = torch.argmax(probs[0], dim=0).cpu().numpy()
        
        # Detectar persona
        person_mask = semantic_mask == 15
        
        # Crear máscara semántica final
        semantic_mask_final = np.zeros_like(semantic_mask)
        semantic_mask_final[road_mask] = 7        # carretera
        semantic_mask_final[sidewalk_mask] = 11   # acera
        semantic_mask_final[person_mask] = 15     # persona
        
        # Crear máscara de instancias solo para personas
        instance_mask = np.zeros_like(semantic_mask)
        person_binary = person_mask.astype(np.uint8)
        num_labels, labels = cv2.connectedComponents(person_binary)
        
        instance_info = {}
        for i in range(1, num_labels):
            y, x = np.where(labels == i)
            if len(x) > 100:  # Filtrar regiones pequeñas
                instance_mask[labels == i] = i
                instance_info[i] = {
                    'class_id': 15,
                    'class_name': 'person',
                    'bbox': [x.min(), y.min(), x.max(), y.max()],
                    'confidence': 0.95,
                    'area': len(x)
                }
        
        return image_rgb, instance_mask, semantic_mask_final, instance_info
        
    except Exception as e:
        print(f"Error al procesar la imagen: {str(e)}")
        return None, None, None, None

def create_panoptic_visualization(image, instance_mask, semantic_mask, instance_info):
    # Colores para la visualización
    class_colors = {
        0: [0, 0, 0],         # background
        7: [128, 64, 128],    # road - gris azulado
        11: [244, 35, 232],   # sidewalk - rosa
        15: [220, 20, 60]     # person - rojo
    }
    
    # Crear visualización base con máscaras semánticas
    vis = np.zeros_like(image)
    
    # Aplicar primero las máscaras semánticas de carretera y acera
    vis[semantic_mask == 7] = class_colors[7]   # carretera
    vis[semantic_mask == 11] = class_colors[11] # acera
    
    # Aplicar las instancias de personas
    for instance_id in instance_info:
        mask = instance_mask == instance_id
        vis[mask] = class_colors[15]  # persona siempre en rojo
    
    # Mezclar con la imagen original
    alpha = 0.6
    final_vis = cv2.addWeighted(image, 1-alpha, vis.astype(np.uint8), alpha, 0)
    
    # Dibujar bounding boxes solo para personas
    for instance_id, info in instance_info.items():
        x1, y1, x2, y2 = info['bbox']
        color = tuple(map(int, class_colors[15]))  # color rojo para personas
        cv2.rectangle(final_vis, (x1, y1), (x2, y2), color, 2)
        
        # Añadir etiqueta
        label = f"Person ({info['confidence']:.2f})"
        cv2.putText(final_vis, label, (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return final_vis

def detect_road_and_sidewalk(image):
    """Detecta carretera y acera usando técnicas de procesamiento de imagen"""
    # Convertir a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Altura y ancho de la imagen
    h, w = gray.shape
    
    # Crear máscaras
    road_mask = np.zeros_like(gray)
    sidewalk_mask = np.zeros_like(gray)
    
    # Región de interés para la carretera (parte inferior central)
    road_mask[int(h*0.6):h, int(w*0.2):int(w*0.8)] = 1
    
    # Región de interés para la acera (bandas laterales)
    sidewalk_mask[int(h*0.4):h, :int(w*0.3)] = 1
    sidewalk_mask[int(h*0.4):h, int(w*0.7):] = 1
    
    return road_mask > 0, sidewalk_mask > 0

def visualize_segmentation(image_path):
    print("Cargando modelo DeepLab...")
    model = deeplabv3_resnet101(pretrained=True)
    model.eval()
    
    print("Procesando imagen...")
    image, instance_mask, semantic_mask, instance_info = process_image(image_path, model)
    if image is None:
        return
    
    panoptic_vis = create_panoptic_visualization(image, instance_mask, semantic_mask, instance_info)
    
    # Visualizar resultados
    plt.figure(figsize=(20, 6))
    
    plt.subplot(131)
    plt.imshow(image)
    plt.title('Imagen Original')
    plt.axis('off')
    
    plt.subplot(132)
    plt.imshow(semantic_mask, cmap='nipy_spectral')
    plt.title('Máscara Semántica')
    plt.axis('off')
    
    plt.subplot(133)
    plt.imshow(panoptic_vis)
    plt.title('Segmentación Panóptica')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Imprimir información solo de las instancias de personas
    print("\nPersonas detectadas:")
    for instance_id, info in instance_info.items():
        print(f"Persona {instance_id}:")
        print(f"  Confianza: {info['confidence']:.2f}")
        print(f"  Área: {info['area']} píxeles")
        print(f"  Bounding Box: {info['bbox']}")
        print()
    
    # Imprimir información de las máscaras semánticas
    unique_classes = np.unique(semantic_mask)
    print("Máscaras semánticas:")
    class_names = {0: "Fondo", 7: "Carretera", 11: "Acera", 15: "Persona"}
    for class_id in unique_classes:
        pixels = np.sum(semantic_mask == class_id)
        percentage = (pixels / semantic_mask.size) * 100
        print(f"{class_names.get(class_id, f'Clase {class_id}')}: {percentage:.2f}% de la imagen")

print("Iniciando segmentación mejorada...")
visualize_segmentation(r'C:\Users\aaron\Documents\Año4\TFG\CityScapesWithPanopticSegm\dataset\images\test\berlin_000009_000019_gtFine_polygons.png')

Iniciando segmentación mejorada...
Cargando modelo DeepLab...


c:\Users\aaron\anaconda3\envs\ia2023\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aaron\anaconda3\envs\ia2023\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Procesando imagen...
Error al procesar la imagen: No se pudo cargar la imagen: C:\Users\aaron\Documents\Año4\TFG\CityScapesWithPanopticSegm\dataset\images\test\berlin_000009_000019_gtFine_polygons.png
